In [1]:
import sys
sys.path.append('./base_model')

In [2]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
import umap
from interface import PolyDisVAE
from base_model import converter

### Configurações

In [3]:
FEATURE_DIR = 'features'
MODEL_PATH = 'model_param/polydis-v1.pt'
BATCH_SIZE = 64
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
from model import DisentangleVAE

In [5]:
model = PolyDisVAE.init_model(device=DEVICE)
if os.path.exists(MODEL_PATH):
    model.load_model(MODEL_PATH)
model.eval()

/workspace/vae-tuning-texutres/interface.py:95: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dic = torch.load(model_path, map_location=self.device)


PolyDisVAE(
  (chd_encoder): ChordEncoder(
    (gru): GRU(36, 1024, batch_first=True, bidirectional=True)
    (linear_mu): Linear(in_features=2048, out_features=256, bias=True)
    (linear_var): Linear(in_features=2048, out_features=256, bias=True)
  )
  (txt_encoder): TextureEncoder(
    (cnn): Sequential(
      (0): Conv2d(1, 10, kernel_size=(4, 12), stride=(4, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0, dilation=1, ceil_mode=False)
    )
    (fc1): Linear(in_features=290, out_features=1000, bias=True)
    (fc2): Linear(in_features=1000, out_features=256, bias=True)
    (gru): GRU(256, 1024, batch_first=True, bidirectional=True)
    (linear_mu): Linear(in_features=2048, out_features=256, bias=True)
    (linear_var): Linear(in_features=2048, out_features=256, bias=True)
  )
  (pnotree_decoder): PianoTreeDecoder(
    (note_embedding): Linear(in_features=135, out_features=128, bias=True)
    (z2dec_hid_linear): Linear(in_features=512, out_fea

In [6]:
def batch_iterator(feature_dir, batch_size):
    files = sorted([f for f in os.listdir(feature_dir) if f.endswith('.npz')])
    for i in range(0, len(files), batch_size):
        batch_files = files[i:i+batch_size]
        piano_rolls = []
        chords = []
        for bf in batch_files:
            data = np.load(os.path.join(feature_dir, bf))
            piano_rolls.append(data['piano_roll'])
            chords.append(data['chord'])
        yield np.stack(piano_rolls), np.stack(chords)

In [8]:
# após carregar o modelo
model.decoder = model.pnotree_decoder
model.rhy_encoder = model.txt_encoder

latents_txt = []
latents_chd = []
recon_losses = []

for pr_batch, chd_batch in batch_iterator(FEATURE_DIR, BATCH_SIZE):
    x = np.stack([converter.target_to_3dtarget(pr) for pr in pr_batch])
    pr_t = torch.tensor(pr_batch, dtype=torch.float32, device=DEVICE)
    chd_t = torch.tensor(chd_batch, dtype=torch.float32, device=DEVICE)
    x_t = torch.tensor(x, dtype=torch.long, device=DEVICE)

    with torch.no_grad():
        out = DisentangleVAE.loss(model, x_t, chd_t, pr_t)
        dist_chd, dist_txt = model.inference_encode(pr_t, chd_t)
    latents_chd.append(dist_chd.mean.cpu().numpy())
    latents_txt.append(dist_txt.mean.cpu().numpy())
    recon_losses.append(out[1].cpu().numpy())

latents_txt = np.concatenate(latents_txt)
latents_chd = np.concatenate(latents_chd)
recon_losses = np.concatenate(recon_losses)

AttributeError: 'PolyDisVAE' object has no attribute 'run'

In [ ]:
reducer = umap.UMAP()
txt_2d = reducer.fit_transform(latents_txt)
chd_2d = reducer.fit_transform(latents_chd)

fig, axs = plt.subplots(1, 2, figsize=(12,5))
sc0 = axs[0].scatter(txt_2d[:,0], txt_2d[:,1], c=recon_losses, cmap='viridis')
axs[0].set_title('Textura (UMAP)')
plt.colorbar(sc0, ax=axs[0])
sc1 = axs[1].scatter(chd_2d[:,0], chd_2d[:,1], c=recon_losses, cmap='viridis')
axs[1].set_title('Acorde (UMAP)')
plt.colorbar(sc1, ax=axs[1])
plt.show()

In [11]:
# após carregar o modelo
model.decoder = model.pnotree_decoder
model.rhy_encoder = model.txt_encoder

# attach functions of DisentangleVAE to this instance
model.run = DisentangleVAE.run.__get__(model, type(model))
model.loss_function = DisentangleVAE.loss_function.__get__(model, type(model))
model.chord_loss = DisentangleVAE.chord_loss.__get__(model, type(model))
model.kl_loss = DisentangleVAE.kl_loss.__get__(model, type(model))
model.loss = DisentangleVAE.loss.__get__(model, type(model))

latents_txt = []
latents_chd = []
recon_losses = []

for pr_batch, chd_batch in batch_iterator(FEATURE_DIR, BATCH_SIZE):
    x = np.stack([converter.target_to_3dtarget(pr) for pr in pr_batch])
    pr_t = torch.tensor(pr_batch, dtype=torch.float32, device=DEVICE)
    chd_t = torch.tensor(chd_batch, dtype=torch.float32, device=DEVICE)
    x_t = torch.tensor(x, dtype=torch.long, device=DEVICE)

    with torch.no_grad():
        out = model.loss(x_t, chd_t, pr_t)
        dist_chd, dist_txt = model.inference_encode(pr_t, chd_t)
    latents_chd.append(dist_chd.mean.cpu().numpy())
    latents_txt.append(dist_txt.mean.cpu().numpy())
    recon_losses.append(out[1].cpu().numpy())

latents_txt = np.concatenate(latents_txt)
latents_chd = np.concatenate(latents_chd)
recon_losses = np.concatenate(recon_losses)

IndexError: shape mismatch: indexing tensors could not be broadcast together with shapes [32768], [22528]